In [ ]:
!unzip -qq '/content/drive/MyDrive/capstone/segmentation_train.zip' -d '/content/train'

In [ ]:
!unzip -qq '/content/drive/MyDrive/capstone/segmentation_val_test.zip'

In [ ]:
import os
import json
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from tqdm import tqdm

In [ ]:
VALID_TOOTH_NUMBERS = {
    11, 12, 13, 14, 15, 16,  # Upper right
    21, 22, 23, 24, 25, 26,  # Upper left
    31, 32, 33, 34, 35, 36,  # Lower left
    41, 42, 43, 44, 45, 46   # Lower right
}

In [ ]:
FDI_TO_INDEX = {tooth_num: idx + 1 for idx, tooth_num in enumerate(sorted(VALID_TOOTH_NUMBERS))}

In [ ]:
FDI_TO_INDEX

{11: 1,
 12: 2,
 13: 3,
 14: 4,
 15: 5,
 16: 6,
 21: 7,
 22: 8,
 23: 9,
 24: 10,
 25: 11,
 26: 12,
 31: 13,
 32: 14,
 33: 15,
 34: 16,
 35: 17,
 36: 18,
 41: 19,
 42: 20,
 43: 21,
 44: 22,
 45: 23,
 46: 24}

In [ ]:
class DentalDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = []
        self.annotations = []

        # FDI 치아 번호를 연속적인 인덱스로 매핑 (1부터 시작)
        self.FDI_TO_INDEX = {tooth_num: idx + 1 for idx, tooth_num in enumerate(sorted(VALID_TOOTH_NUMBERS))}

        # 데이터셋 로드
        subdirs = ['1.right', '2.front', '3.left', '4.upper', '5.lower']
        for subdir in subdirs:
            image_path = os.path.join(image_dir, subdir)
            label_path = os.path.join(label_dir, subdir)

            if not os.path.exists(image_path) or not os.path.exists(label_path):
                continue

            for label_file in os.listdir(label_path):
                if label_file.endswith(".json"):
                    label_filepath = os.path.join(label_path, label_file)
                    try:
                        with open(label_filepath, 'r', encoding='utf-8') as f:
                            annotation = json.load(f)
                            image_filepath = annotation["image_filepath"].replace("\\", "/")
                            image_name = os.path.basename(image_filepath)
                            corrected_path = os.path.join(self.image_dir,
                                                          os.path.dirname(image_filepath).split('/')[-1],
                                                          image_name)

                            # 유효한 치아 번호 필터링
                            valid_teeth = [tooth for tooth in annotation["tooth"]
                                           if tooth["teeth_num"] in VALID_TOOTH_NUMBERS]

                            if valid_teeth:
                                self.image_files.append(corrected_path)
                                self.annotations.append(valid_teeth)
                    except (json.JSONDecodeError, KeyError) as e:
                        print(f"Error loading {label_file}: {e}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # 이미지 로드
        img_path = self.image_files[idx]
        img = Image.open(img_path).convert("RGB")

        # 어노테이션 처리
        teeth = self.annotations[idx]
        num_teeth = len(teeth)

        # 마스크와 박스 생성
        masks = []
        boxes = []
        labels = []

        for tooth in teeth:
            # 세그멘테이션 포인트로부터 바이너리 마스크 생성
            points = np.array(tooth["segmentation"])
            mask = np.zeros((img.height, img.width), dtype=np.uint8)
            if len(points) > 2:
                from PIL import ImageDraw
                mask_img = Image.new('L', (img.width, img.height), 0)
                ImageDraw.Draw(mask_img).polygon(points.flatten().tolist(), outline=1, fill=1)
                mask = np.array(mask_img)

            # 마스크에서 바운딩 박스 추출
            if mask.any():
                pos = np.where(mask)
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                boxes.append([xmin, ymin, xmax, ymax])
                masks.append(mask)

                # 치아 번호를 매핑된 인덱스로 변환
                tooth_label = tooth["teeth_num"]
                if tooth_label in self.FDI_TO_INDEX:
                    labels.append(self.FDI_TO_INDEX[tooth_label])
                else:
                    print(f"Warning: Invalid tooth number {tooth_label} encountered.")

        # 텐서 형식으로 변환
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(np.array(masks), dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_teeth,), dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'masks': masks,
            'image_id': image_id,
            'area': area,
            'iscrowd': iscrowd
        }

        if self.transform is not None:
            img = self.transform(img)

        return img, target


In [ ]:
def get_model(num_classes):
    model = maskrcnn_resnet50_fpn_v2(weights="DEFAULT")

    # Replace the pre-trained box predictor
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Replace the pre-trained mask predictor
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                      hidden_layer,
                                                      num_classes)

    return model

In [ ]:

class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:

@torch.no_grad()
def evaluate(model, data_loader, device):
    model.eval()
    losses = AverageMeter()

    for images, targets in tqdm(data_loader, desc="Validating"):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        total_loss = sum(loss for loss in loss_dict.values())
        losses.update(total_loss.item(), len(images))

    model.train()
    return losses.avg

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, device, num_epochs=10, save_dir='checkpoints'):
    os.makedirs(save_dir, exist_ok=True)
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        train_losses = AverageMeter()

        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            losses.backward()
            optimizer.step()

            train_losses.update(losses.item(), len(images))

        val_loss = evaluate(model, val_loader, device)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Training Loss: {train_losses.avg:.4f}")
        print(f"Validation Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_losses.avg,
                'val_loss': val_loss,
            }, os.path.join(save_dir, 'best_model.pth'))

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_losses.avg,
            'val_loss': val_loss,
        }, os.path.join(save_dir, f'checkpoint_epoch_{epoch+1}.pth'))

In [ ]:
weight_decay = 0.01
batch_size=8
lr = 0.0001
num_epochs = 10

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Dataset paths
train_image_dir = "/content/train/1.원천데이터"
train_label_dir = "/content/train/2.라벨링데이터"
val_image_dir = "/content/Validation/1.원천데이터"
val_label_dir = "/content/Validation/2.라벨링데이터"

# Transform
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

# Create datasets
train_dataset = DentalDataset(train_image_dir, train_label_dir, transform=transform)
val_dataset = DentalDataset(val_image_dir, val_label_dir, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                        num_workers=4, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                      num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

# Print dataset sizes
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")


Training dataset size: 58055
Validation dataset size: 7256


In [ ]:
# 모델 초기화
num_classes = len(VALID_TOOTH_NUMBERS) + 1  # 24 teeth + background
model = get_model(num_classes)
model.to(device)

# AdamW 옵티마이저 사용
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=lr, weight_decay=weight_decay)
# 학습 시작
train_model(model, train_loader, val_loader, optimizer, device, num_epochs=num_epochs)


Epoch 1/10:  76%|███████▌  | 5506/7257 [1:00:12<19:30,  1.50it/s]

# YOLO-Seg

### train 옮기기

In [ ]:
import os
import shutil

# 상위 디렉토리 경로
source_dir = "/content/train/1.원천데이터"  # 폴더 안에 하위 폴더들이 있는 경로
target_dir = "/content/dataset/images/train"  # 파일을 옮길 대상 폴더 경로

# 대상 폴더가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 하위 폴더 순회
for subdir in os.listdir(source_dir):
    subdir_path = os.path.join(source_dir, subdir)
    if os.path.isdir(subdir_path):
        # 하위 폴더 안의 모든 파일 이동
        for file_name in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file_name)
            if os.path.isfile(file_path):
                shutil.move(file_path, target_dir)
                print(f"Moved: {file_path} -> {target_dir}")

print("모든 파일 이동이 완료되었습니다.")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Moved: /content/train/1.원천데이터/3.left/left_5258.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_14870.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_10846.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_7239.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_15867.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_5171.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_10824.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_5854.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_16803.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_17269.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.left/left_14985.png -> /content/dataset/images/train
Moved: /content/train/1.원천데이터/3.l

In [ ]:
import os
import shutil

# 상위 디렉토리 경로
source_dir = "/content/train/2.라벨링데이터"  # 폴더 안에 하위 폴더들이 있는 경로
target_dir = "/content/dataset/labels/train"  # 파일을 옮길 대상 폴더 경로

# 대상 폴더가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 하위 폴더 순회
for subdir in os.listdir(source_dir):
    subdir_path = os.path.join(source_dir, subdir)
    if os.path.isdir(subdir_path):
        # 하위 폴더 안의 모든 파일 이동
        for file_name in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file_name)
            if os.path.isfile(file_path):
                shutil.move(file_path, target_dir)
                #print(f"Moved: {file_path} -> {target_dir}")

print("모든 파일 이동이 완료되었습니다.")


모든 파일 이동이 완료되었습니다.


### val 옮기기

In [ ]:
import os
import shutil

# 상위 디렉토리 경로
source_dir = "/content/Validation/1.원천데이터"  # 폴더 안에 하위 폴더들이 있는 경로
target_dir = "/content/dataset/images/val"  # 파일을 옮길 대상 폴더 경로

# 대상 폴더가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 하위 폴더 순회
for subdir in os.listdir(source_dir):
    subdir_path = os.path.join(source_dir, subdir)
    if os.path.isdir(subdir_path):
        # 하위 폴더 안의 모든 파일 이동
        for file_name in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file_name)
            if os.path.isfile(file_path):
                shutil.move(file_path, target_dir)
                #print(f"Moved: {file_path} -> {target_dir}")

print("모든 파일 이동이 완료되었습니다.")


모든 파일 이동이 완료되었습니다.


In [ ]:
import os
import shutil

# 상위 디렉토리 경로
source_dir = "/content/Validation/2.라벨링데이터"  # 폴더 안에 하위 폴더들이 있는 경로
target_dir = "/content/dataset/labels/val"  # 파일을 옮길 대상 폴더 경로

# 대상 폴더가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 하위 폴더 순회
for subdir in os.listdir(source_dir):
    subdir_path = os.path.join(source_dir, subdir)
    if os.path.isdir(subdir_path):
        # 하위 폴더 안의 모든 파일 이동
        for file_name in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file_name)
            if os.path.isfile(file_path):
                shutil.move(file_path, target_dir)
                #print(f"Moved: {file_path} -> {target_dir}")

print("모든 파일 이동이 완료되었습니다.")


모든 파일 이동이 완료되었습니다.


### test 옮기기

In [ ]:
import os
import shutil

# 상위 디렉토리 경로
source_dir = "/content/test/1.원천데이터"  # 폴더 안에 하위 폴더들이 있는 경로
target_dir = "/content/dataset/images/test"  # 파일을 옮길 대상 폴더 경로

# 대상 폴더가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 하위 폴더 순회
for subdir in os.listdir(source_dir):
    subdir_path = os.path.join(source_dir, subdir)
    if os.path.isdir(subdir_path):
        # 하위 폴더 안의 모든 파일 이동
        for file_name in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file_name)
            if os.path.isfile(file_path):
                shutil.move(file_path, target_dir)
                #print(f"Moved: {file_path} -> {target_dir}")

print("모든 파일 이동이 완료되었습니다.")


모든 파일 이동이 완료되었습니다.


In [ ]:
import os
import shutil

# 상위 디렉토리 경로
source_dir = "/content/test/2.라벨링데이터"  # 폴더 안에 하위 폴더들이 있는 경로
target_dir = "/content/dataset/labels/test"  # 파일을 옮길 대상 폴더 경로

# 대상 폴더가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 하위 폴더 순회
for subdir in os.listdir(source_dir):
    subdir_path = os.path.join(source_dir, subdir)
    if os.path.isdir(subdir_path):
        # 하위 폴더 안의 모든 파일 이동
        for file_name in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file_name)
            if os.path.isfile(file_path):
                shutil.move(file_path, target_dir)
                #print(f"Moved: {file_path} -> {target_dir}")

print("모든 파일 이동이 완료되었습니다.")


모든 파일 이동이 완료되었습니다.


### dataset yolo형식으로 바꾸기

### class_id x_center y_center width height is_decayed complexity_id

In [ ]:
import json
import os
from PIL import Image

# FDI 클래스 매핑 정의
FDI_CLASS_MAPPING = {
    11: 0, 12: 1, 13: 2, 14: 3, 15: 4, 16: 5,
    21: 6, 22: 7, 23: 8, 24: 9, 25: 10, 26: 11,
    31: 12, 32: 13, 33: 14, 34: 15, 35: 16, 36: 17,
    41: 18, 42: 19, 43: 20, 44: 21, 45: 22, 46: 23
}

def convert_segmentation_to_yolo(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # 이미지 파일 경로 수정
    image_name = os.path.basename(data["image_filepath"])

    # JSON 파일의 경로에 따라 이미지 경로 설정
    label_dir = os.path.dirname(json_file)
    split = os.path.basename(label_dir)  # train, val, or test
    image_filepath = os.path.join('/content/dataset/images', split, image_name)

    # 이미지 열기 및 크기 측정
    image = Image.open(image_filepath)
    image_width, image_height = image.size

    # 텍스트 파일 경로 설정 (JSON 파일이 있는 곳에 저장)
    text_file_path = os.path.splitext(json_file)[0] + '.txt'

    # 텍스트 파일 열기
    with open(text_file_path, 'w') as txt_file:
        for tooth in data['tooth']:
            tooth_num = tooth['teeth_num']
            segmentation = tooth['segmentation']

            # 클래스 인덱스 매핑
            class_index = FDI_CLASS_MAPPING.get(tooth_num, -1)  # 매핑되지 않으면 -1로 설정

            # YOLO 형식 준비
            points = []
            for point in segmentation:
                # 좌표 정규화
                x_norm = point[0] / image_width
                y_norm = point[1] / image_height
                points.append(f"{x_norm} {y_norm}")

            # 점들을 문자열로 결합
            points_str = ' '.join(points)

            # YOLO 형식으로 텍스트 파일에 쓰기
            if class_index != -1:  # 유효한 클래스 인덱스인 경우에만 기록
                yolo_line = f"{class_index} {points_str}\n"
                txt_file.write(yolo_line)

def process_all_json_files(base_dir):
    # train, val, test 디렉토리
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(base_dir, split)

        # 모든 JSON 파일 탐색
        for root, _, files in os.walk(split_dir):
            for file in files:
                if file.endswith('.json'):
                    json_file_path = os.path.join(root, file)
                    convert_segmentation_to_yolo(json_file_path)

# 사용 예시
process_all_json_files('/content/dataset/labels/')


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from PIL import Image
import numpy as np
import cv2
# 이미지 파일과 라벨 파일 경로 지정
image_path ='/content/dataset/images/test/front_16229.png'  # 예시 이미지 파일 경로
#label_path = '/content/dataset/labels/test/front_16229.txt'  # 예시 라벨 파일 경로

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
height, width, _ = image.shape

# 세그멘테이션 데이터가 저장된 txt 파일 로드
segmentation_data_path = '/content/dataset/labels/test/front_16229.txt'

# 데이터 파일 읽기
with open(segmentation_data_path, 'r') as file:
    data = file.readlines()


# 이미지에 세그멘테이션 다각형 그리기
fig, ax = plt.subplots()
ax.imshow(image)

# 각 세그멘테이션 결과에 대해 반복
for line in data:
    # 데이터 파싱
    split_line = line.strip().split()
    obj_id = split_line[0]
    coordinates = np.array(split_line[1:], dtype=float)

    # 좌표를 x, y 쌍으로 재배열
    coordinates = coordinates.reshape(-1, 2)

    # 이미지 크기에 맞춰 좌표 조정
    x = coordinates[:, 0] * width
    y = coordinates[:, 1] * height

    # 다각형 그리기
    ax.plot(np.append(x, x[0]), np.append(y, y[0]), 'r-', linewidth=2)

ax.set_xlim(0, width)
ax.set_ylim(0, height)
ax.set_aspect('equal')
plt.gca().invert_yaxis()  # y축 뒤집기

# 이미지 파일로 저장
plt.axis('off')  # 축 정보 제거
plt.savefig('segmentation_on_image.png', bbox_inches='tight', pad_inches=0)
plt.close()



In [ ]:
!unzip -qq '/content/yolo_form_label (1).zip' -d '/content/labels'

### json 파일 지우기

In [ ]:
import os
import glob

# JSON label 디렉토리 경로 설정
label_dir = "dataset/labels"

# 하위 폴더 순회 (train, val, test)
for split in ['train', 'val', 'test']:
    json_files = glob.glob(os.path.join(label_dir, split, "*.json"))

    # 모든 .json 파일 삭제
    for json_file in json_files:
        os.remove(json_file)
        print(f"Deleted: {json_file}")

print("모든 .json 파일이 삭제되었습니다.")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Deleted: dataset/labels/test/right_19209.json
Deleted: dataset/labels/test/front_17087.json
Deleted: dataset/labels/test/upper_31562.json
Deleted: dataset/labels/test/lower_42034.json
Deleted: dataset/labels/test/left_18039.json
Deleted: dataset/labels/test/lower_42885.json
Deleted: dataset/labels/test/upper_29747.json
Deleted: dataset/labels/test/right_19438.json
Deleted: dataset/labels/test/lower_40472.json
Deleted: dataset/labels/test/lower_40063.json
Deleted: dataset/labels/test/lower_41760.json
Deleted: dataset/labels/test/upper_31139.json
Deleted: dataset/labels/test/upper_30488.json
Deleted: dataset/labels/test/upper_31749.json
Deleted: dataset/labels/test/upper_30671.json
Deleted: dataset/labels/test/right_18887.json
Deleted: dataset/labels/test/upper_29855.json
Deleted: dataset/labels/test/right_18847.json
Deleted: dataset/labels/test/lower_41718.json
Deleted: dataset/labels/test/upper_29814.json
Deleted: dataset/labels/test/left_19011.json


### label 압축코드

In [ ]:
import shutil
import os

# 압축할 폴더 경로
folder_to_zip = "/content/dataset/labels"
# 생성될 압축 파일의 경로 및 이름 (확장자 제외)
output_zip_file = "./yolo_form_label"

# 폴더 압축 (.zip 형식)
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

print(f"압축 파일이 생성되었습니다: {output_zip_file}.zip")


압축 파일이 생성되었습니다: ./yolo_form_label.zip


train.txt

In [ ]:
import os
import glob

# 이미지 디렉토리 설정 (train 폴더만 사용)
train_dir = "/content/dataset/images/train"
val_dir = "/content/dataset/images/val"
test_dir = "/content/dataset/images/test"

# 출력 파일 경로
output_file = "/content/dataset/train.txt"

# `train.txt` 파일 생성
with open(output_file, 'w') as f:
    # `train` 폴더의 모든 PNG 파일의 절대 경로 가져오기
    image_files = glob.glob(os.path.join(train_dir, "*.png"))

    for image_path in image_files:
        # 절대 경로로 기록
        f.write(f"{os.path.abspath(image_path)}\n")

print(f"'{output_file}' 파일이 생성되었습니다.")

output_file = "/content/dataset/val.txt"

# `val.txt` 파일 생성
with open(output_file, 'w') as f:
    # `train` 폴더의 모든 PNG 파일의 절대 경로 가져오기
    image_files = glob.glob(os.path.join(val_dir, "*.png"))

    for image_path in image_files:
        # 절대 경로로 기록
        f.write(f"{os.path.abspath(image_path)}\n")

print(f"'{output_file}' 파일이 생성되었습니다.")

output_file = "/content/dataset/test.txt"

# `val.txt` 파일 생성
with open(output_file, 'w') as f:
    # `train` 폴더의 모든 PNG 파일의 절대 경로 가져오기
    image_files = glob.glob(os.path.join(test_dir, "*.png"))

    for image_path in image_files:
        # 절대 경로로 기록
        f.write(f"{os.path.abspath(image_path)}\n")

print(f"'{output_file}' 파일이 생성되었습니다.")



'/content/dataset/train.txt' 파일이 생성되었습니다.
'/content/dataset/val.txt' 파일이 생성되었습니다.
'/content/dataset/test.txt' 파일이 생성되었습니다.


Yolo 학습

In [ ]:
pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11s-seg.pt")

model.train(data="/content/dataset/data.yaml",
            epochs=12,
            batch=-1,
            imgsz=640,
            )

Ultralytics 8.3.31 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=/content/dataset/data.yaml, epochs=12, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning /content/dataset/labels/train.cache... 58055 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58055/58055 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA A100-SXM4-40GB) 39.56G total, 0.13G reserved, 0.11G allocated, 39.32G free


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    10091576       35.64         0.422         50.77           nan        (1, 3, 640, 640)                    list
    10091576       71.28         0.952         43.58           nan        (2, 3, 640, 640)                    list
    10091576       142.6         1.760         43.56           nan        (4, 3, 640, 640)                    list
    10091576       285.1         3.379         48.34           nan        (8, 3, 640, 640)                    list
    10091576       570.3         6.549         53.79           nan       (16, 3, 640, 640)                    list
    10091576        1141        12.843         62.75           nan       (32, 3, 640, 640)                    list
    10091576        2281        25.296         94.14           nan       (64, 3, 640, 640)                    list
AutoBatch: Using batch-size 59 for CUDA:0 23.63G/39.56G (60%) ✅


...

      12/12      20.4G     0.2709     0.3709     0.2433     0.8125        708        640: 100%|██████████| 984/984 [10:45<00:00,  1.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [01:14<00:00,  1.20s/it]


                   all       7256      89270      0.881      0.915      0.954      0.921      0.881      0.915      0.955      0.904

12 epochs completed in 3.610 hours.
Optimizer stripped from runs/segment/train6/weights/last.pt, 20.5MB
Optimizer stripped from runs/segment/train6/weights/best.pt, 20.5MB

Validating runs/segment/train6/weights/best.pt...
Ultralytics 8.3.31 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11s-seg summary (fused): 265 layers, 10,076,104 parameters, 0 gradients, 35.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [01:24<00:00,  1.37s/it]


                   all       7256      89270       0.89      0.923      0.955      0.922       0.89      0.923      0.956      0.904
               tooth11       3811       3811      0.888      0.915      0.961      0.947      0.888      0.915      0.961      0.936
               tooth12       3710       3710        0.9      0.923      0.965      0.937        0.9      0.923      0.965      0.914
               tooth13       3771       3771      0.894      0.925      0.966      0.942      0.894      0.925      0.966      0.921
               tooth14       3795       3795      0.896        0.9      0.956       0.91      0.897        0.9      0.957      0.884
               tooth15       2476       2476      0.858      0.878      0.932      0.895      0.858      0.878      0.932      0.869
               tooth16       2812       2812       0.83      0.881       0.92      0.888       0.83      0.881       0.92      0.876
               tooth21       3818       3818      0.848      0.936   

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x792789fc5f60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034

In [ ]:
model = YOLO("/content/runs/segment/train/weights/best.pt")  # load a custom model

# Predict with the model
source = '/content/gum5.jpg'


results = model.predict(
    source,
    #conf=0.2,
    save=True,
    project="result",
    name="inference",
    exist_ok=True
    )

print("추론 결과가 'inference_output' 폴더에 저장되었습니다.")


image 1/1 /content/gum5.jpg: 384x640 1 tooth11, 1 tooth12, 1 tooth13, 1 tooth14, 2 tooth15s, 1 tooth21, 2 tooth22s, 1 tooth23, 1 tooth31, 1 tooth32, 1 tooth33, 1 tooth34, 1 tooth41, 1 tooth42, 1 tooth43, 1 tooth44, 16.3ms
Speed: 3.0ms preprocess, 16.3ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to result/inference
추론 결과가 'inference_output' 폴더에 저장되었습니다.
